La empresa Instagram almacena información de usuarios en un RDD de la forma (id_usuario,nickname,pais,fecha_alta) y en otro RDD almacena información de las publicaciones como (id_publicacion, id_usuario,id_foto,comentario,fecha)

a) Encontrar el top 10 de hashtags que más aparecen y que fueron publicados únicamente por usuarios de Argentina.

b) Construir y cachear un RDD con los pares de usuarios (usuario_1,usuario_2) que tienen hashtags publicados en común.

In [18]:
import pyspark
import re

In [117]:
posts = [(16747,9838,116,'Every man #chabon has his price','2019/03/11'),
(36922,1438,126,'Youve got to #locura ask yourself one question Do I feel lucky? Well do ya punk','2019/03/29'),
(11386,1592,108,'Like two peas #locura #chabon in a pod','2019/01/28'),
(39223,1082,137,'Dont put the cart before the horse','2019/03/02'),
(8296,2639,106,'Nostalgia isnt what it used to be','2019/03/02'),
(16116,3829,137,'Shes drop dead #chabon gorgeous','2019/01/11'),
(20129,7677,146,'Too many #dromedario cooks spoil the broth','2019/01/04'),
(44960,8276,140,'If you cant beat them, join them','2019/01/28'),
(6832,3536,148,'I like turtles','2019/01/11'),
(44692,6709,150,'Ask no questions and hear no lies','2019/02/27'),
(12374,1328,122,'A bird in the #locura hand is #chabon worth two in the bush','2019/03/01'),
(50730,3964,148,'Youll #dromedario poke your eye out','2019/01/09'),
(47049,1880,122,'You can #bobsponge lead a horse #chabon to water but you cant make him drink','2019/02/16'),
(6320,5624,127,'Two heads are better than one','2019/01/23'),
(37911,1444,132,'Im going to make #chabon him an offer he cant refuse','2019/02/17'),
(44080,4014,113,'A sight for sore eyes','2019/02/12'),
(57898,5635,118,'Theres #bobsponge no place #dromedario like home','2019/01/01'),
(10721,5123,142,'Absence #dromedario makes the heart #locura grow fonder','2019/01/24'),
(57053,6084,116,'Youre barking up the wrong tree','2019/02/01'),
(11695,2885,141,'Old habits die hard','2019/02/07'),
(22564,6943,145,'No pain no gain','2019/03/09'),
(21307,1813,136,'Variety is the spice of #chabon #locura life','2019/03/10'),
(49413,1554,103,'Would you jump off #bobsponge a bridge if all your friends were doing it','2019/03/13'),
(55044,3591,129,'A friend in need is a friend indeed','2019/04/01'),
(38938,8740,137,'Dont cry over spilled milk','2019/01/01'),
(31140,9651,106,'The needs of the #bobsponge many #dromedario outweigh the needs #locura of the few','2019/02/05'),
(44856,6027,136,'Because I said so, thats #dromedario why','2019/03/13'),
(4517,9673,103,'The Tribe has spoken','2019/02/15'),
(28579,4650,150,'The pen is mightier #bobsponge than the sword','2019/01/18'),
(52683,6501,119,'Leave well enough #dromedario alone','2019/01/20')]

users = [
    (9838,'Raindrop','argentina','5/14/2018'),
    (1438,'Admiral','uruguay','9/24/2018'),
    (1592,'Mustache','uruguay','7/7/2018'),
    (1082,'Bean','argentina','2/9/2018'),
    (2639,'Fatty','uruguay','4/18/2018'),
    (3829,'Queenie','uruguay','1/23/2018'),
    (7677,'Sugar','argentina','8/7/2018'),
    (8276,'Righty','argentina','10/10/2018'),
    (3536,'Fellow','argentina','9/3/2018'),
    (6709,'Dearest','argentina','8/6/2018'),
    (1328,'Salt','uruguay','5/11/2018'),
    (3964,'Belch','argentina','8/28/2018'),
    (1880,'Boo Bear','uruguay','5/23/2018'),
    (5624,'Bello','argentina','2/6/2018'),
    (1444,'Chain','argentina','11/1/2018'),
    (4014,'Doc','argentina','7/4/2018'),
    (5635,'Dirty Harry','argentina','4/5/2018'),
    (5123,'Jackrabbit','argentina','5/31/2018'),
    (6084,'Foxy','uruguay','11/4/2018'),
    (2885,'Cruella','argentina','11/25/2018'),
    (6943,'Guy','uruguay','10/1/2018'),
    (1813,'Buck','chile','8/4/2018'),
    (1554,'Peppermint','chile','2/15/2018'),
    (3591,'Headlights','chile','1/6/2018'),
    (8740,'Slim','chile','10/22/2018'),
    (9651,'Skunk','chile','8/18/2018'),
    (6027,'Blondie','chile','9/16/2018'),
    (9673,'Bandit','argentina','4/11/2018'),
    (4650,'Sport','argentina','11/7/2018'),
    (6501,'Pookie','argentina','11/7/2018')
]

In [118]:
usersRdd = sc.parallelize(users)
postsRdd = sc.parallelize(posts)

In [119]:
# primero "listo" los hashtags
postsRdd = postsRdd.flatMap(lambda x: [(word, x[1]) for word in x[3].split()])\
.filter(lambda x: '#' in x[0])\
.map(lambda x: (x[1],(x[0],1)))

postsRdd.collect()

[(9838, ('#chabon', 1)),
 (1438, ('#locura', 1)),
 (1592, ('#locura', 1)),
 (1592, ('#chabon', 1)),
 (3829, ('#chabon', 1)),
 (7677, ('#dromedario', 1)),
 (1328, ('#locura', 1)),
 (1328, ('#chabon', 1)),
 (3964, ('#dromedario', 1)),
 (1880, ('#bobsponge', 1)),
 (1880, ('#chabon', 1)),
 (1444, ('#chabon', 1)),
 (5635, ('#bobsponge', 1)),
 (5635, ('#dromedario', 1)),
 (5123, ('#dromedario', 1)),
 (5123, ('#locura', 1)),
 (1813, ('#chabon', 1)),
 (1813, ('#locura', 1)),
 (1554, ('#bobsponge', 1)),
 (9651, ('#bobsponge', 1)),
 (9651, ('#dromedario', 1)),
 (9651, ('#locura', 1)),
 (6027, ('#dromedario', 1)),
 (4650, ('#bobsponge', 1)),
 (6501, ('#dromedario', 1))]

In [120]:
# me quedo con los usuarios que NO son de argentina (porque quiero los hashtags que son UNICAMENTE de argentina)
usersRdd = usersRdd.filter(lambda x: x[2] != 'argentina')\
.map(lambda x: (x[0], 0))

usersRdd.collect()

[(1438, 0),
 (1592, 0),
 (2639, 0),
 (3829, 0),
 (1328, 0),
 (1880, 0),
 (6084, 0),
 (6943, 0),
 (1813, 0),
 (1554, 0),
 (3591, 0),
 (8740, 0),
 (9651, 0),
 (6027, 0)]

In [121]:
hashtagsInvalidos = usersRdd.join(postsRdd).collect()
hashtagsInvalidos

[(1328, (0, ('#locura', 1))),
 (1328, (0, ('#chabon', 1))),
 (1554, (0, ('#bobsponge', 1))),
 (9651, (0, ('#bobsponge', 1))),
 (9651, (0, ('#dromedario', 1))),
 (9651, (0, ('#locura', 1))),
 (3829, (0, ('#chabon', 1))),
 (1813, (0, ('#chabon', 1))),
 (1813, (0, ('#locura', 1))),
 (1592, (0, ('#locura', 1))),
 (1592, (0, ('#chabon', 1))),
 (1880, (0, ('#bobsponge', 1))),
 (1880, (0, ('#chabon', 1))),
 (6027, (0, ('#dromedario', 1))),
 (1438, (0, ('#locura', 1)))]

In [122]:
hashtagsInvalidos = [x[1][1] for x in hashtagsInvalidos]
hashtagsInvalidos

[('#locura', 1),
 ('#chabon', 1),
 ('#bobsponge', 1),
 ('#bobsponge', 1),
 ('#dromedario', 1),
 ('#locura', 1),
 ('#chabon', 1),
 ('#chabon', 1),
 ('#locura', 1),
 ('#locura', 1),
 ('#chabon', 1),
 ('#bobsponge', 1),
 ('#chabon', 1),
 ('#dromedario', 1),
 ('#locura', 1)]

In [131]:
publicaciones = postsRdd.filter(lambda x: x[1] not in hashtagsInvalidos)
publicaciones.collect()

[]

In [132]:
publicaciones.map(lambda x: (x[1][0], x[1][1])).reduceByKey(lambda x,y: x+y).takeOrdered(10, lambda x: -x[1])

[]

In [111]:
# todo lo que sigue esta MAL
# primero filtro los users de argentina
#usersRdd = usersRdd.filter(lambda x: x[2] == 'argentina')\
#.map(lambda x: (x[0], 0))

#usersRdd.collect()

In [112]:
# mapeo para poder joinear con users
#postsRdd = postsRdd.map(lambda x: (x[1], x[3]))
#postsRdd.collect()

In [113]:
# inner join porque me piden posts que sean de usuarios de Arg.
#argPostsRdd = postsRdd.join(usersRdd)
#argPostsRdd.collect()

In [114]:
#argPostsRdd = argPostsRdd.filter(lambda x: '#' in x[1][0])
#argPostsRdd.collect()

In [115]:
# necesito quedarme con los hashtags
#def getHashtags(x):
#    tags = []
#    tags += re.findall(r'[#@][^\s#@]+', x[1][0])
#    return (x[0], tags)
    
#hashtagsRdd = argPostsRdd.map(getHashtags)

#hashtagsRdd.collect()

In [116]:
# ahora mapeo con el hashtag como clave y les pongo un contador
#hashtagsRdd.flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).takeOrdered(10, lambda x: -x[1])